# Product Recommendation System with Hybrid Search

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/etl/recommendation_system.ipynb)

Build a product recommendation engine that combines vector similarity with metadata filtering. Users get semantically similar products filtered by category, price range, and availability.

In [ ]:
!pip install -q qdrant-client sentence-transformers

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import (
    PointStruct, VectorParams, Distance,
    PayloadSchemaType, Filter, FieldCondition, MatchValue, Range,
)
from sentence_transformers import SentenceTransformer

In [ ]:
client = QdrantClient(":memory:")
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
client.create_collection(
    collection_name="products",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

# Index payload fields for fast filtering
for field, schema in [
    ("category", PayloadSchemaType.KEYWORD),
    ("price", PayloadSchemaType.FLOAT),
    ("in_stock", PayloadSchemaType.KEYWORD),
]:
    client.create_payload_index(
        collection_name="products",
        field_name=field,
        field_schema=schema,
    )

In [ ]:
# Sample product catalog
products = [
    {"name": "Wireless Noise-Canceling Headphones", "category": "electronics", "price": 299.99, "in_stock": "yes"},
    {"name": "Bluetooth Earbuds with ANC", "category": "electronics", "price": 149.99, "in_stock": "yes"},
    {"name": "Studio Monitor Headphones", "category": "electronics", "price": 199.99, "in_stock": "no"},
    {"name": "Portable Bluetooth Speaker", "category": "electronics", "price": 79.99, "in_stock": "yes"},
    {"name": "Running Shoes with Cushioning", "category": "sports", "price": 129.99, "in_stock": "yes"},
    {"name": "Trail Running Shoes Waterproof", "category": "sports", "price": 159.99, "in_stock": "yes"},
    {"name": "Yoga Mat Premium Non-Slip", "category": "sports", "price": 49.99, "in_stock": "yes"},
    {"name": "Resistance Bands Set", "category": "sports", "price": 29.99, "in_stock": "yes"},
    {"name": "Mechanical Keyboard RGB", "category": "electronics", "price": 149.99, "in_stock": "yes"},
    {"name": "Ergonomic Office Chair", "category": "furniture", "price": 449.99, "in_stock": "yes"},
    {"name": "Standing Desk Adjustable", "category": "furniture", "price": 599.99, "in_stock": "no"},
    {"name": "Desk Lamp LED Adjustable", "category": "furniture", "price": 39.99, "in_stock": "yes"},
]

points = [
    PointStruct(
        id=i,
        vector=model.encode(p["name"]).tolist(),
        payload=p,
    )
    for i, p in enumerate(products)
]

client.upsert(collection_name="products", points=points)
print(f"Indexed {len(products)} products")

In [ ]:
def recommend(query: str, category: str = None, max_price: float = None, in_stock_only: bool = True, limit: int = 5):
    """Get product recommendations with optional filters."""
    conditions = []
    if category:
        conditions.append(FieldCondition(key="category", match=MatchValue(value=category)))
    if max_price:
        conditions.append(FieldCondition(key="price", range=Range(lte=max_price)))
    if in_stock_only:
        conditions.append(FieldCondition(key="in_stock", match=MatchValue(value="yes")))

    query_filter = Filter(must=conditions) if conditions else None
    query_vec = model.encode(query).tolist()

    response = client.query_points(
        collection_name="products",
        query=query_vec,
        query_filter=query_filter,
        limit=limit,
    )
    return response.points

In [ ]:
# Scenario 1: User wants headphones under $200
print("=== Headphones under $200 ===")
results = recommend("high quality headphones for music", category="electronics", max_price=200)
for r in results:
    print(f"  {r.score:.4f} | ${r.payload['price']} | {r.payload['name']}")

# Scenario 2: User wants workout gear
print("\n=== Workout gear ===")
results = recommend("equipment for home workouts", category="sports")
for r in results:
    print(f"  {r.score:.4f} | ${r.payload['price']} | {r.payload['name']}")

# Scenario 3: Similar to a product (recommendation by example)
print("\n=== Similar to 'Running Shoes' ===")
results = recommend("Running Shoes with Cushioning", in_stock_only=True)
for r in results:
    print(f"  {r.score:.4f} | [{r.payload['category']}] ${r.payload['price']} | {r.payload['name']}")